In [1]:
import os
import timm
import torch
import open_clip
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from transformers import AutoModel
from timm import create_model
import torchvision.transforms as T
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [2]:

file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePalooza Data/Benchmarking/Beetlepalooza_beetles_trait_weather.csv"

df = pd.read_csv(file_path)

df.head(3)


,ImageFilePath,Genus,ScientificName,Length,Width,TempMin,TempMax,Precipitation,Rain,Snowfall,WindGust,WindSpeed,TempAvg
0,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius,Chlaenius aestivus,1.117336,0.637281,26.6,18.9,2.0,2.0,0.0,34.6,14.6,22.75
1,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius,Chlaenius aestivus,1.068345,0.627180,26.6,18.9,2.0,2.0,0.0,34.6,14.6,22.75
2,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius,Chlaenius aestivus,1.107754,0.627256,26.6,18.9,2.0,2.0,0.0,34.6,14.6,22.75


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = create_model("convnext_base", pretrained=True).to(device)


preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [4]:
def extract_features(image_path) : 
    
    image = Image.open(image_path).convert("RGB")
    
    image_tensor = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        features = model.forward_features(image_tensor).mean(dim=[2, 3])

    return features.cpu().numpy()
    

In [5]:
df['Embedding'] = [extract_features(img) for img in tqdm(df['ImageFilePath'], desc='Extracting Embeddings')]

X = np.vstack(df['Embedding'].values)
numeric_cols = df.loc[:, 'ScientificName':].select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [col for col in numeric_cols if col != 'ScientificName']
X = np.hstack((X, df[numeric_cols].values))

label_encoder = LabelEncoder()

y = label_encoder.fit_transform(df['ScientificName'])


Extracting Embeddings: 100%|██████████| 11399/11399 [23:24<00:00,  8.12it/s] 


In [6]:
df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [7]:
seed = 99

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),   
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}

for name, model in models.items():
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T

/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


NaiveBayes                | Acc: 79.61% | Prec: 83.20% | Rec: 79.61% | F1: 80.55% | Bal Acc: 73.11% | MCC: 0.7843


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


LogisticRegression        | Acc: 96.32% | Prec: 95.86% | Rec: 96.32% | F1: 95.85% | Bal Acc: 79.47% | MCC: 0.9606


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


NearestNeighbor           | Acc: 86.27% | Prec: 84.84% | Rec: 86.27% | F1: 84.50% | Bal Acc: 58.57% | MCC: 0.8527
MLP_Baseline              | Acc: 96.49% | Prec: 96.04% | Rec: 96.49% | F1: 96.04% | Bal Acc: 82.63% | MCC: 0.9624


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [8]:
test_df = test_df.assign(**{f"Pred_{name}": label_encoder.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,Genus,ScientificName,Length,Width,TempMin,TempMax,Precipitation,Rain,Snowfall,WindGust,WindSpeed,TempAvg,Embedding,Pred_NaiveBayes,Pred_LogisticRegression,Pred_NearestNeighbor,Pred_MLP_Baseline
1766,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Synuchus,Synuchus impunctatus,0.530811,0.242519,27.3,18.9,18.7,18.7,0.0,38.5,11.6,23.10,"[[2.2748895, -1.1932064, -1.9985443, 2.1603513...",Cymindis cribricollis,Synuchus impunctatus,Synuchus impunctatus,Synuchus impunctatus
2629,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Agonum,Agonum punctiforme,0.457965,0.243925,29.7,16.6,0.0,0.0,0.0,36.4,18.1,23.15,"[[1.7991288, -1.5654267, -1.2066966, 2.005974,...",Agonum punctiforme,Agonum punctiforme,Synuchus impunctatus,Synuchus impunctatus


In [9]:
test_df.to_csv("/users/PAS2136/rayees/CV4A - MultiModality/Full/ConvNeXt-traits-weather.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/CV4A - MultiModality/Full/ConvNeXt-traits-weather-metrics.csv", index=False)